#BART-Large-CNN Text Summarization

The facebook/bart-large-cnn model uses a sophisticated approach to determine what text is important in a document. Here's a breakdown of how it works:

##Architecture and Training
BART (Bidirectional and Auto-Regressive Transformers) combines the bidirectional encoding of **BERT** with the autoregressive decoding of **GPT**. It's essentially a sequence-to-sequence model with:

**Encoder:** Processes the entire input text bidirectionally (looking at words in context of both what comes before and after)
**Decoder:** Generates the summary one token at a time

The "CNN" in the model name refers to the CNN/Daily Mail dataset it was fine-tuned on, which contains news articles paired with human-written summaries.

##How It Determines Important Information
BART-large-CNN identifies important content through several mechanisms:

**1. Attention Mechanisms**
The model uses self-attention to weigh the importance of different words relative to each other. This allows it to:

Identify which words are most referenced by other parts of the text
Understand relationships between entities and concepts
Focus on sentences with high information density

**2. Learned Patterns from News Articles**
Since it was trained on CNN/Daily Mail news articles, the model has learned journalistic patterns:

Important information tends to appear in the opening paragraphs
Key information usually answers who, what, when, where, why questions
Names, dates, places, and numeric facts often deserve emphasis

**3. Content Density Recognition**
The model has learned to identify sentences with high information content by recognizing:

Higher concentration of named entities (people, places, organizations)
Statistical importance of terms (similar to TF-IDF patterns)
Sentences that contain new information not redundant with other parts

**4. Document Structure Understanding**
BART can recognize structural importance:

Section headings and topic sentences usually contain important content
Conclusion sentences often summarize key points
Repetition of concepts across the document signals importance

##The Summarization Process

**Encoding:** The document is processed through the encoder, which builds semantic representations of each token in context

**Importance Weighting:** Through attention mechanisms, the model assigns importance weights to various parts of the text

**Content Selection:** The decoder selects the most salient information to include

**Generation:** The decoder generates a fluent summary that preserves the key points

Unlike extractive summarization (which just pulls out existing sentences), BART performs abstractive summarization, meaning it can rephrase and combine information in new ways.
The model doesn't simply extract the most important sentences - it understands concepts and can express them in different words, making the summaries more concise and fluid than simple extraction would allow.

##Implementation

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline
from typing import List, Dict, Any, Optional
import textwrap
import numpy as np

In [ ]:
class TextSummarizer:
    def __init__(self, model_name: str = "facebook/bart-large-cnn"):
        """
        Initialize the text summarizer with a Hugging Face model.

        Args:
            model_name (str): Name of the Hugging Face model to use.
                Default: "facebook/bart-large-cnn" (good balance of quality and speed)
                Other options:
                - "t5-small" (faster, lower quality)
                - "google/pegasus-xsum" (news-focused)
                - "philschmid/bart-large-cnn-samsum" (optimized for dialogues)
        """
        print(f"Loading model: {model_name} (this may take a minute on first run)")
        try:
            self.tokenizer = AutoTokenizer.from_pretrained(model_name)
            self.model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

            # For advanced summarization
            self.summarization_pipeline = pipeline(
                "summarization",
                model=model_name,
                device=0 if torch.cuda.is_available() else -1  # Use GPU if available
            )

            self.device = "cuda" if torch.cuda.is_available() else "cpu"
            print(f"Model loaded successfully. Using device: {self.device}")

            # Store model name for reference
            self.model_name = model_name

        except Exception as e:
            print(f"Error loading model: {e}")
            print("Trying to load a smaller model as fallback...")
            try:
                # Fallback to a smaller model
                fallback_model = "t5-small"
                self.tokenizer = AutoTokenizer.from_pretrained(fallback_model)
                self.model = AutoModelForSeq2SeqLM.from_pretrained(fallback_model)
                self.summarization_pipeline = pipeline(
                    "summarization",
                    model=fallback_model,
                    device=0 if torch.cuda.is_available() else -1
                )
                self.model_name = fallback_model
                print(f"Fallback model loaded: {fallback_model}")
            except Exception as e2:
                raise RuntimeError(f"Failed to load both primary and fallback models: {e2}")

    def summarize(self, text: str, max_length: int = 150, min_length: int = 30,
                  style: str = "concise", preserve_key_points: bool = True) -> str:
        """
        Summarize the given text using the loaded model.

        Args:
            text (str): The text to summarize.
            max_length (int): Maximum token length for the summary.
            min_length (int): Minimum token length for the summary.
            style (str): Style of summary ("concise", "detailed", "bullet-points").
            preserve_key_points (bool): Whether to try to preserve key information.

        Returns:
            str: The generated summary.
        """
        # Adjust parameters based on style
        if style == "concise":
            # Keep defaults, shorter summary
            pass
        elif style == "detailed":
            # Increase length for more details
            max_length = max(max_length * 2, 250)
            min_length = max(min_length * 2, 100)
        elif style == "bullet-points":
            # Will be post-processed into bullet points
            pass

        # For more control, directly set parameters for pipeline
        num_beams = 4
        if preserve_key_points:
            # Using more beams and lower temperature helps preserve key information
            num_beams = 6
            temperature = 0.7
        else:
            temperature = 1.0

        # Handle long texts by chunking if needed
        max_input_length = self.tokenizer.model_max_length
        input_ids = self.tokenizer.encode(text, truncation=False)

        if len(input_ids) > max_input_length:
            # Text is too long, need to chunk it
            return self._summarize_long_text(text, max_length, min_length, num_beams, temperature)

        # Generate summary
        summary = self.summarization_pipeline(
            text,
            max_length=max_length,
            min_length=min_length,
            num_beams=num_beams,
            temperature=temperature,
            early_stopping=True
        )

        result = summary[0]['summary_text']

        # Post-process for bullet points if requested
        if style == "bullet-points":
            # Convert to bullet points by identifying key sentences
            sentences = [s.strip() for s in result.split('.') if s.strip()]
            bullet_points = "\n".join([f"• {s}." for s in sentences])
            return bullet_points

        return result

    def _summarize_long_text(self, text: str, max_length: int, min_length: int,
                             num_beams: int, temperature: float) -> str:
        """
        Handle summarization of texts longer than the model's maximum input length.
        Uses a chunking approach to process long documents.

        Args:
            text (str): The long text to summarize.
            max_length (int): Maximum token length for the final summary.
            min_length (int): Minimum token length for the final summary.
            num_beams (int): Number of beams for beam search.
            temperature (float): Temperature for generation.

        Returns:
            str: The generated summary.
        """
        # Split text into sentences
        sentences = [s.strip() + '.' for s in text.replace('\n', ' ').split('.') if s.strip()]

        # Calculate max tokens for model
        max_tokens = self.tokenizer.model_max_length - 50  # Reserve tokens for generation

        # Create chunks that fit within model's context window
        chunks = []
        current_chunk = []
        current_length = 0

        for sentence in sentences:
            sentence_tokens = len(self.tokenizer.encode(sentence))

            if current_length + sentence_tokens > max_tokens:
                # This chunk is full, start a new one
                if current_chunk:  # Make sure we don't add empty chunks
                    chunks.append(' '.join(current_chunk))
                current_chunk = [sentence]
                current_length = sentence_tokens
            else:
                current_chunk.append(sentence)
                current_length += sentence_tokens

        # Add the last chunk if it's not empty
        if current_chunk:
            chunks.append(' '.join(current_chunk))

        # Summarize each chunk
        chunk_summaries = []
        for i, chunk in enumerate(chunks):
            print(f"Summarizing chunk {i+1}/{len(chunks)}...")
            chunk_summary = self.summarization_pipeline(
                chunk,
                max_length=max(30, max_length // len(chunks)),  # Scale based on chunks
                min_length=min(min_length // len(chunks), 20),  # Ensure some minimum
                num_beams=num_beams,
                temperature=temperature
            )
            chunk_summaries.append(chunk_summary[0]['summary_text'])

        # If we have multiple chunk summaries, summarize them together
        if len(chunk_summaries) > 1:
            combined_summary = " ".join(chunk_summaries)

            # Check if the combined summary is still too long
            if len(self.tokenizer.encode(combined_summary)) > max_tokens:
                # Recursively summarize the combined summaries
                return self._summarize_long_text(
                    combined_summary, max_length, min_length, num_beams, temperature
                )
            else:
                # Final summary of the combined chunk summaries
                final_summary = self.summarization_pipeline(
                    combined_summary,
                    max_length=max_length,
                    min_length=min_length,
                    num_beams=num_beams,
                    temperature=temperature
                )
                return final_summary[0]['summary_text']
        else:
            # If we only had one chunk, its summary is the final summary
            return chunk_summaries[0]

    def batch_summarize(self, texts: List[str], **kwargs) -> List[str]:
        """
        Summarize multiple texts at once.

        Args:
            texts (List[str]): List of texts to summarize.
            **kwargs: Additional arguments passed to summarize method.

        Returns:
            List[str]: List of summaries.
        """
        return [self.summarize(text, **kwargs) for text in texts]

    def evaluate_summary(self, original_text: str, summary: str) -> Dict[str, float]:
        """
        Evaluate the quality of a summary compared to the original text using ROUGE metrics.

        Args:
            original_text (str): The original text.
            summary (str): The summary to evaluate.

        Returns:
            Dict[str, float]: Dictionary with evaluation metrics.
        """
        try:
            # Try to import rouge
            from rouge import Rouge
            rouge = Rouge()

            # Calculate ROUGE scores
            scores = rouge.get_scores(summary, original_text)

            # Extract metrics
            evaluation = {
                "rouge-1_precision": scores[0]["rouge-1"]["p"],
                "rouge-1_recall": scores[0]["rouge-1"]["r"],
                "rouge-1_f1": scores[0]["rouge-1"]["f"],
                "rouge-2_f1": scores[0]["rouge-2"]["f"],
                "rouge-l_f1": scores[0]["rouge-l"]["f"]
            }

            # Overall score (weighted average of F1 scores)
            evaluation["overall_score"] = (
                0.4 * evaluation["rouge-1_f1"] +
                0.3 * evaluation["rouge-2_f1"] +
                0.3 * evaluation["rouge-l_f1"]
            )

            return evaluation

        except ImportError:
            # If rouge is not installed, return basic word overlap analysis
            print("Rouge package not found. Using basic evaluation.")
            return self._basic_evaluation(original_text, summary)

    def _basic_evaluation(self, original_text: str, summary: str) -> Dict[str, float]:
        """Simple evaluation when Rouge is not available."""
        orig_words = set(original_text.lower().split())
        summary_words = set(summary.lower().split())

        # Calculate word overlap
        overlap = len(orig_words.intersection(summary_words))
        precision = overlap / len(summary_words) if summary_words else 0
        recall = overlap / len(orig_words) if orig_words else 0
        f1 = 2 * precision * recall / (precision + recall) if (precision + recall) else 0

        return {
            "word_precision": precision,
            "word_recall": recall,
            "word_f1": f1,
            "overall_score": f1
        }

    def get_available_models(self) -> List[Dict[str, str]]:
        """
        Returns a list of recommended free models for summarization.

        Returns:
            List[Dict[str, str]]: List of model information.
        """
        return [
            {
                "name": "facebook/bart-large-cnn",
                "description": "Good balance between quality and speed, trained on CNN/DM dataset",
                "size": "400MB",
                "strengths": "General purpose summarization, news articles"
            },
            {
                "name": "t5-small",
                "description": "Smaller and faster model, good for resource-constrained environments",
                "size": "242MB",
                "strengths": "Speed, works on lower-end hardware"
            },
            {
                "name": "google/pegasus-xsum",
                "description": "State-of-the-art model for extreme summarization (very concise)",
                "size": "2.2GB",
                "strengths": "Very concise summaries, news articles"
            },
            {
                "name": "philschmid/bart-large-cnn-samsum",
                "description": "Fine-tuned for dialogue summarization",
                "size": "400MB",
                "strengths": "Conversations, chat logs, meetings"
            },
            {
                "name": "sshleifer/distilbart-cnn-12-6",
                "description": "Distilled version of BART, faster with similar quality",
                "size": "306MB",
                "strengths": "Good balance of speed and quality"
            }
        ]



In [ ]:
# Example usage
if __name__ == "__main__":
    # Sample text to summarize
    sample_text = """
    An ancient jawbone dredged from the Taiwanese seabed has revealed new insights into the appearance and sweeping geographic range of an enigmatic human species called the Denisovans.

    The fossil was discovered by fishers trawling the Penghu Channel off Taiwan and is thought to be the most complete fossil that has been genetically identified as Denisovan. The male individual, who lived at least 10,000 years ago, had a strong jaw and very large, powerful molars.

    “From a tooth or a small bone fragment, there’s the mystery of their appearance,” said Prof Enrico Cappellini, of the University of Copenhagen, a co-senior author on the paper. A Denisovan jaw discovered in Tibet had begun to fill in this picture, and the latest discovery adds to the evidence of a prominent jaw with huge teeth.

    “Now we have a richer image,” Cappellini said. “Of course it would be good to have a skull and the rest of the skeleton, but it’s a step forward.”

    The fossil has been dated to one of two glacial periods when the channel is known to have been above sea level, either between 10,000 and 70,000 years ago or between 130,000 and 190,000 years ago.
    The discovery reveals an impressive geographic range for the ancient species, which lived at the same time as – and interbred with – modern humans and Neanderthals.

    The first Denisovan fossils, identified through analysis of ancient DNA, came from a cave in Siberia and comprised just a finger fragment and a few teeth. Since then, further discoveries show Denisovans also weathered the incredibly harsh conditions of the high-altitude Tibetan plateau, where temperatures can plunge to -30C. By contrast, in south-east Asia they would have lived alongside water buffaloes in a balmy climate.

    “These are climate and environmental conditions that are quite different,” Cappellini said. “The cold environment in Siberia, high altitude in Tibet. We cannot infer anything of their cognitive abilities … but they had an ability to adapt to environments that are quite diverse.”

    """

    try:
        # Print available models
        print("Available summarization models:")
        summarizer = TextSummarizer()  # Initialize with default model
        models = summarizer.get_available_models()
        for i, model in enumerate(models):
            print(f"{i+1}. {model['name']} - {model['description']} ({model['size']})")

        print("\nUsing default model:", summarizer.model_name)

        # Generate a concise summary
        print("\nGenerating concise summary...")
        concise_summary = summarizer.summarize(
            sample_text,
            max_length=75,
            min_length=30,
            style="concise"
        )

        # Generate a detailed summary
        print("Generating detailed summary...")
        detailed_summary = summarizer.summarize(
            sample_text,
            style="detailed"
        )

        # Generate bullet points
        print("Generating bullet points...")
        bullet_points = summarizer.summarize(
            sample_text,
            style="bullet-points"
        )

        # Evaluate the concise summary
        print("Evaluating summary...")
        evaluation = summarizer.evaluate_summary(sample_text, concise_summary)

        # Print results
        print("\n--- ORIGINAL TEXT ---")
        print(textwrap.fill(sample_text, width=80))

        print("\n--- CONCISE SUMMARY ---")
        print(textwrap.fill(concise_summary, width=80))

        print("\n--- DETAILED SUMMARY ---")
        print(textwrap.fill(detailed_summary, width=80))

        print("\n--- BULLET POINTS ---")
        print(bullet_points)

        print("\n--- EVALUATION ---")
        for metric, score in evaluation.items():
            print(f"{metric}: {score:.3f}")

    except Exception as e:
        print(f"Error: {e}")
        print("\nTroubleshooting:")
        print("1. Make sure you have installed the required packages:")
        print("   pip install torch transformers")
        print("2. For evaluation metrics, install rouge:")
        print("   pip install rouge")
        print("3. If you have limited RAM, try using a smaller model:")
        print("   summarizer = TextSummarizer('t5-small')")

Available summarization models:
Loading model: facebook/bart-large-cnn (this may take a minute on first run)


Device set to use cpu


Model loaded successfully. Using device: cpu
1. facebook/bart-large-cnn - Good balance between quality and speed, trained on CNN/DM dataset (400MB)
2. t5-small - Smaller and faster model, good for resource-constrained environments (242MB)
3. google/pegasus-xsum - State-of-the-art model for extreme summarization (very concise) (2.2GB)
4. philschmid/bart-large-cnn-samsum - Fine-tuned for dialogue summarization (400MB)
5. sshleifer/distilbart-cnn-12-6 - Distilled version of BART, faster with similar quality (306MB)

Using default model: facebook/bart-large-cnn

Generating concise summary...
Generating detailed summary...
Generating bullet points...
Evaluating summary...
Rouge package not found. Using basic evaluation.

--- ORIGINAL TEXT ---
     An ancient jawbone dredged from the Taiwanese seabed has revealed new
insights into the appearance and sweeping geographic range of an enigmatic human
species called the Denisovans.      The fossil was discovered by fishers
trawling the Penghu Cha

#Summarize pdf/word doc

In [ ]:
pip install PyPDF2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 4.4 MB/s eta 0:00:00


In [ ]:
pip install python-docx

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/244.3 kB 4.5 MB/s eta 0:00:00


In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline
from typing import List, Dict, Any, Optional, Union, IO
import textwrap
import numpy as np
import os
from pathlib import Path
import tempfile

In [ ]:
class DocumentTextExtractor:
    """Extract text from various document formats like PDF and Word."""

    def __init__(self):
        """Initialize the document text extractor."""
        # Check for required libraries
        self._check_dependencies()

    def _check_dependencies(self):
        """Check if required libraries are installed and provide installation instructions if not."""
        missing_libs = []

        # Check for PDF support
        try:
            import PyPDF2
        except ImportError:
            missing_libs.append("PyPDF2")

        # Check for Word document support
        try:
            import docx
        except ImportError:
            missing_libs.append("python-docx")

        # Print installation instructions if libraries are missing
        if missing_libs:
            print("Missing required libraries for document processing:")
            print(f"pip install {' '.join(missing_libs)}")
            print("Installing these libraries will enable PDF and Word document support.")

    def extract_text(self, document_path: Union[str, Path, IO]) -> str:
        """
        Extract text from a document file (PDF or Word).

        Args:
            document_path: Path to the document or file-like object

        Returns:
            str: Extracted text content

        Raises:
            ValueError: If the document format is unsupported
            ImportError: If required libraries are not installed
        """
        # Handle file path as string or Path object
        if isinstance(document_path, (str, Path)):
            file_path = str(document_path)
            file_ext = os.path.splitext(file_path)[1].lower()

            # Extract text based on file extension
            if file_ext == '.pdf':
                return self._extract_from_pdf(file_path)
            elif file_ext in ['.docx', '.doc']:
                return self._extract_from_word(file_path)
            else:
                try:
                    # Try to read as plain text
                    with open(file_path, 'r', encoding='utf-8') as f:
                        return f.read()
                except Exception:
                    raise ValueError(f"Unsupported document format: {file_ext}")

        # Handle file-like objects
        else:
            # Create a temporary file to save the content
            with tempfile.NamedTemporaryFile(delete=False) as temp_file:
                temp_path = temp_file.name
                # Read and write content
                document_path.seek(0)
                temp_file.write(document_path.read())

            try:
                # Try different extractors
                try:
                    return self._extract_from_pdf(temp_path)
                except Exception:
                    try:
                        return self._extract_from_word(temp_path)
                    except Exception:
                        # Try as plain text
                        with open(temp_path, 'r', encoding='utf-8') as f:
                            return f.read()
            finally:
                # Clean up temp file
                if os.path.exists(temp_path):
                    os.unlink(temp_path)

    def _extract_from_pdf(self, pdf_path: str) -> str:
        """Extract text from a PDF file."""
        try:
            import PyPDF2
        except ImportError:
            raise ImportError("PyPDF2 is required for PDF support. Install with: pip install PyPDF2")

        text = ""
        with open(pdf_path, 'rb') as file:
            pdf_reader = PyPDF2.PdfReader(file)
            for page_num in range(len(pdf_reader.pages)):
                page = pdf_reader.pages[page_num]
                text += page.extract_text() + "\n\n"

        return text

    def _extract_from_word(self, docx_path: str) -> str:
        """Extract text from a Word document."""
        try:
            import docx
        except ImportError:
            raise ImportError("python-docx is required for Word document support. Install with: pip install python-docx")

        doc = docx.Document(docx_path)
        text = []

        for para in doc.paragraphs:
            text.append(para.text)

        return '\n'.join(text)


In [ ]:
class TextSummarizer:
    """A text summarization model using free, locally-run models with document support."""

    def __init__(self, model_name: str = "facebook/bart-large-cnn"):
        """
        Initialize the text summarizer with a Hugging Face model.

        Args:
            model_name (str): Name of the Hugging Face model to use.
                Default: "facebook/bart-large-cnn" (good balance of quality and speed)
                Other options:
                - "t5-small" (faster, lower quality)
                - "google/pegasus-xsum" (news-focused)
                - "philschmid/bart-large-cnn-samsum" (optimized for dialogues)
        """
        print(f"Loading model: {model_name} (this may take a minute on first run)")
        try:
            self.tokenizer = AutoTokenizer.from_pretrained(model_name)
            self.model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

            # For advanced summarization
            self.summarization_pipeline = pipeline(
                "summarization",
                model=model_name,
                device=0 if torch.cuda.is_available() else -1  # Use GPU if available
            )

            self.device = "cuda" if torch.cuda.is_available() else "cpu"
            print(f"Model loaded successfully. Using device: {self.device}")

            # Store model name for reference
            self.model_name = model_name

            # Initialize document extractor
            self.document_extractor = DocumentTextExtractor()

        except Exception as e:
            print(f"Error loading model: {e}")
            print("Trying to load a smaller model as fallback...")
            try:
                # Fallback to a smaller model
                fallback_model = "t5-small"
                self.tokenizer = AutoTokenizer.from_pretrained(fallback_model)
                self.model = AutoModelForSeq2SeqLM.from_pretrained(fallback_model)
                self.summarization_pipeline = pipeline(
                    "summarization",
                    model=fallback_model,
                    device=0 if torch.cuda.is_available() else -1
                )
                self.model_name = fallback_model
                print(f"Fallback model loaded: {fallback_model}")

                # Initialize document extractor
                self.document_extractor = DocumentTextExtractor()
            except Exception as e2:
                raise RuntimeError(f"Failed to load both primary and fallback models: {e2}")

    def summarize_document(self, document_path: Union[str, Path, IO], **kwargs) -> str:
        """
        Summarize a document file (PDF, Word, or text).

        Args:
            document_path: Path to the document or file-like object
            **kwargs: Additional arguments passed to summarize method

        Returns:
            str: The generated summary
        """
        try:
            # Extract text from document
            print("Extracting text from document...")
            text = self.document_extractor.extract_text(document_path)

            # Summarize the extracted text
            print(f"Summarizing document ({len(text.split())} words)...")
            return self.summarize(text, **kwargs)
        except ImportError as e:
            print(f"Error: {e}")
            return "Required libraries not installed. See error message above for installation instructions."
        except Exception as e:
            print(f"Error processing document: {e}")
            return f"Failed to summarize document: {str(e)}"

    def summarize(self, text: str, max_length: int = 150, min_length: int = 30,
                  style: str = "concise", preserve_key_points: bool = True) -> str:
        """
        Summarize the given text using the loaded model.

        Args:
            text (str): The text to summarize.
            max_length (int): Maximum token length for the summary.
            min_length (int): Minimum token length for the summary.
            style (str): Style of summary ("concise", "detailed", "bullet-points").
            preserve_key_points (bool): Whether to try to preserve key information.

        Returns:
            str: The generated summary.
        """
        # Adjust parameters based on style
        if style == "concise":
            # Keep defaults, shorter summary
            pass
        elif style == "detailed":
            # Increase length for more details
            max_length = max(max_length * 2, 250)
            min_length = max(min_length * 2, 100)
        elif style == "bullet-points":
            # Will be post-processed into bullet points
            pass

        # For more control, directly set parameters for pipeline
        num_beams = 4
        if preserve_key_points:
            # Using more beams and lower temperature helps preserve key information
            num_beams = 6
            temperature = 0.7
        else:
            temperature = 1.0

        # Handle long texts by chunking if needed
        max_input_length = self.tokenizer.model_max_length
        input_ids = self.tokenizer.encode(text, truncation=False)

        if len(input_ids) > max_input_length:
            # Text is too long, need to chunk it
            return self._summarize_long_text(text, max_length, min_length, num_beams, temperature)

        # Generate summary
        summary = self.summarization_pipeline(
            text,
            max_length=max_length,
            min_length=min_length,
            num_beams=num_beams,
            temperature=temperature,
            early_stopping=True
        )

        result = summary[0]['summary_text']

        # Post-process for bullet points if requested
        if style == "bullet-points":
            # Convert to bullet points by identifying key sentences
            sentences = [s.strip() for s in result.split('.') if s.strip()]
            bullet_points = "\n".join([f"• {s}." for s in sentences])
            return bullet_points

        return result

    def _summarize_long_text(self, text: str, max_length: int, min_length: int,
                             num_beams: int, temperature: float) -> str:
        """
        Handle summarization of texts longer than the model's maximum input length.
        Uses a chunking approach to process long documents.

        Args:
            text (str): The long text to summarize.
            max_length (int): Maximum token length for the final summary.
            min_length (int): Minimum token length for the final summary.
            num_beams (int): Number of beams for beam search.
            temperature (float): Temperature for generation.

        Returns:
            str: The generated summary.
        """
        # Split text into sentences
        sentences = [s.strip() + '.' for s in text.replace('\n', ' ').split('.') if s.strip()]

        # Calculate max tokens for model
        max_tokens = self.tokenizer.model_max_length - 50  # Reserve tokens for generation

        # Create chunks that fit within model's context window
        chunks = []
        current_chunk = []
        current_length = 0

        for sentence in sentences:
            sentence_tokens = len(self.tokenizer.encode(sentence))

            if current_length + sentence_tokens > max_tokens:
                # This chunk is full, start a new one
                if current_chunk:  # Make sure we don't add empty chunks
                    chunks.append(' '.join(current_chunk))
                current_chunk = [sentence]
                current_length = sentence_tokens
            else:
                current_chunk.append(sentence)
                current_length += sentence_tokens

        # Add the last chunk if it's not empty
        if current_chunk:
            chunks.append(' '.join(current_chunk))

        # Summarize each chunk
        chunk_summaries = []
        for i, chunk in enumerate(chunks):
            print(f"Summarizing chunk {i+1}/{len(chunks)}...")
            chunk_summary = self.summarization_pipeline(
                chunk,
                max_length=max(30, max_length // len(chunks)),  # Scale based on chunks
                min_length=min(min_length // len(chunks), 20),  # Ensure some minimum
                num_beams=num_beams,
                temperature=temperature
            )
            chunk_summaries.append(chunk_summary[0]['summary_text'])

        # If we have multiple chunk summaries, summarize them together
        if len(chunk_summaries) > 1:
            combined_summary = " ".join(chunk_summaries)

            # Check if the combined summary is still too long
            if len(self.tokenizer.encode(combined_summary)) > max_tokens:
                # Recursively summarize the combined summaries
                return self._summarize_long_text(
                    combined_summary, max_length, min_length, num_beams, temperature
                )
            else:
                # Final summary of the combined chunk summaries
                final_summary = self.summarization_pipeline(
                    combined_summary,
                    max_length=max_length,
                    min_length=min_length,
                    num_beams=num_beams,
                    temperature=temperature
                )
                return final_summary[0]['summary_text']
        else:
            # If we only had one chunk, its summary is the final summary
            return chunk_summaries[0]

    def batch_summarize(self, texts: List[str], **kwargs) -> List[str]:
        """
        Summarize multiple texts at once.

        Args:
            texts (List[str]): List of texts to summarize.
            **kwargs: Additional arguments passed to summarize method.

        Returns:
            List[str]: List of summaries.
        """
        return [self.summarize(text, **kwargs) for text in texts]

    def batch_summarize_documents(self, document_paths: List[Union[str, Path]], **kwargs) -> List[str]:
        """
        Summarize multiple documents at once.

        Args:
            document_paths: List of paths to documents
            **kwargs: Additional arguments passed to summarize method

        Returns:
            List[str]: List of summaries
        """
        return [self.summarize_document(doc_path, **kwargs) for doc_path in document_paths]

    def evaluate_summary(self, original_text: str, summary: str) -> Dict[str, float]:
        """
        Evaluate the quality of a summary compared to the original text using ROUGE metrics.

        Args:
            original_text (str): The original text.
            summary (str): The summary to evaluate.

        Returns:
            Dict[str, float]: Dictionary with evaluation metrics.
        """
        try:
            # Try to import rouge
            from rouge import Rouge
            rouge = Rouge()

            # Calculate ROUGE scores
            scores = rouge.get_scores(summary, original_text)

            # Extract metrics
            evaluation = {
                "rouge-1_precision": scores[0]["rouge-1"]["p"],
                "rouge-1_recall": scores[0]["rouge-1"]["r"],
                "rouge-1_f1": scores[0]["rouge-1"]["f"],
                "rouge-2_f1": scores[0]["rouge-2"]["f"],
                "rouge-l_f1": scores[0]["rouge-l"]["f"]
            }

            # Overall score (weighted average of F1 scores)
            evaluation["overall_score"] = (
                0.4 * evaluation["rouge-1_f1"] +
                0.3 * evaluation["rouge-2_f1"] +
                0.3 * evaluation["rouge-l_f1"]
            )

            return evaluation

        except ImportError:
            # If rouge is not installed, return basic word overlap analysis
            print("Rouge package not found. Using basic evaluation.")
            return self._basic_evaluation(original_text, summary)

    def _basic_evaluation(self, original_text: str, summary: str) -> Dict[str, float]:
        """Simple evaluation when Rouge is not available."""
        orig_words = set(original_text.lower().split())
        summary_words = set(summary.lower().split())

        # Calculate word overlap
        overlap = len(orig_words.intersection(summary_words))
        precision = overlap / len(summary_words) if summary_words else 0
        recall = overlap / len(orig_words) if orig_words else 0
        f1 = 2 * precision * recall / (precision + recall) if (precision + recall) else 0

        return {
            "word_precision": precision,
            "word_recall": recall,
            "word_f1": f1,
            "overall_score": f1
        }

    def get_available_models(self) -> List[Dict[str, str]]:
        """
        Returns a list of recommended free models for summarization.

        Returns:
            List[Dict[str, str]]: List of model information.
        """
        return [
            {
                "name": "facebook/bart-large-cnn",
                "description": "Good balance between quality and speed, trained on CNN/DM dataset",
                "size": "400MB",
                "strengths": "General purpose summarization, news articles"
            },
            {
                "name": "t5-small",
                "description": "Smaller and faster model, good for resource-constrained environments",
                "size": "242MB",
                "strengths": "Speed, works on lower-end hardware"
            },
            {
                "name": "google/pegasus-xsum",
                "description": "State-of-the-art model for extreme summarization (very concise)",
                "size": "2.2GB",
                "strengths": "Very concise summaries, news articles"
            },
            {
                "name": "philschmid/bart-large-cnn-samsum",
                "description": "Fine-tuned for dialogue summarization",
                "size": "400MB",
                "strengths": "Conversations, chat logs, meetings"
            },
            {
                "name": "sshleifer/distilbart-cnn-12-6",
                "description": "Distilled version of BART, faster with similar quality",
                "size": "306MB",
                "strengths": "Good balance of speed and quality"
            }
        ]



In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
if __name__ == "__main__":
    # Initialize the summarizer
    summarizer = TextSummarizer()

    # Example: Summarize a PDF document
    # (Uncomment and modify with your PDF path)

    docx_path = "/content/drive/My Drive/SumText_1.docx"
    if os.path.exists(docx_path):
        print(f"\nSummarizing Word document: {docx_path}")
        docx_summary = summarizer.summarize_document(
            docx_path,
            style="detailed"
        )
        print("\n--- WORD DOCUMENT SUMMARY ---")
        print(docx_summary)
    else:
        print(f"Word document not found: {docx_path}")


Loading model: facebook/bart-large-cnn (this may take a minute on first run)


Device set to use cpu


Model loaded successfully. Using device: cpu

Summarizing Word document: /content/drive/My Drive/SumText_1.docx
Extracting text from document...
Summarizing document (678 words)...

--- WORD DOCUMENT SUMMARY ---
US and Ukrainian officials met on Friday to discuss White House proposals for a minerals deal. Donald Trump wants Kyiv to hand over its natural resources as ‘payback’ in return for weapons delivered by the previous Biden administration. Latest US draft is more ‘maximalist’ than the original version from February, which proposed giving Washington $500bn worth of rare metals, as well as oil and gas. Ukraine cut off the supply of gas when its five-year contract with the Russian state energy company Gazprom expired.
